## Grid Search

In [ ]:
from multiprocessing import Process
from models.wordle_base_15k import run_simulations as rl_base
from models.wordle_cluster_15k import run_simulations as rl_cluster
from models.wordle_cluster_2k import run_simulations as rl_cluster_2
import pandas as pd

# Our hyper-parameters to search on
learning_rates = [0.1, 0.01, 0.001]    
exploration_rates = [0.5, 0.6, 0.7, 0.8, 0.9]
shrinkage_factors = [0.5, 0.6, 0.7, 0.8, 0.9]
num_of_clusters = [6, 7, 8, 9, 10]

# Set to True to rerun grid_search
run_grid_search = False

# Grid-search function for RL-base 15k
def func_1():
    df_1 = pd.DataFrame(columns=['learning_rate', 'exploration_rate', 'shrinkage_factor', 'time_taken', 'average_guesses', 'win_rate'])

    for i, alpha in enumerate(learning_rates):
        for j, epsilon in enumerate(exploration_rates):
            for k, gamma in enumerate(shrinkage_factors):
                print(f'Running epoch {i}, {j}, {k}')
                time_taken, average_guesses, win_rate, guesses = rl_base(learning_rate=alpha, exploration_rate=epsilon, shrinkage_factor=gamma, num_simulations=100)
                df_1.loc[len(df_1)] = [alpha, epsilon, gamma, time_taken, average_guesses, win_rate]
                
    df_1.to_csv('grid_search_results/results_base.csv', index=False)
    df_1 = df_1.sort_values(by=['win_rate', 'average_guesses', 'time_taken'], ascending=[False, True, True])
    print(df_1.iloc[0])

# Grid-search function for RL with clustering 15k
def func_2():
    df_2 = pd.DataFrame(columns=['learning_rate', 'exploration_rate', 'shrinkage_factor', 'num_of_clusters', 'time_taken', 'average_guesses', 'win_rate'])

    for i, alpha in enumerate(learning_rates):
        for j, epsilon in enumerate(exploration_rates):
            for k, gamma in enumerate(shrinkage_factors):
                for l, num_clusters in enumerate(num_of_clusters):
                    print(f'Running epoch {i}, {j}, {k}, {l}')
                    time_taken, average_guesses, win_rate, guesses = rl_cluster(learning_rate=alpha, exploration_rate=epsilon, shrinkage_factor=gamma, number_of_cluster=num_clusters, num_simulations=100)
                    df_2.loc[len(df_2)] = [alpha, epsilon, gamma, num_clusters, time_taken, average_guesses, win_rate]

    df_2.to_csv('grid_search_results/results_cluster.csv', index=False)
    df_2 = df_2.sort_values(by=['win_rate', 'average_guesses', 'time_taken'], ascending=[False, True, True])
    print(df_2.iloc[0])

# Grid-search function for RL with clustering 2k
def func_3():
    df_3 = pd.DataFrame(columns=['learning_rate', 'exploration_rate', 'shrinkage_factor', 'num_of_clusters', 'time_taken', 'average_guesses', 'win_rate'])

    for i, alpha in enumerate(learning_rates):
        for j, epsilon in enumerate(exploration_rates):
            for k, gamma in enumerate(shrinkage_factors):
                for l,num_clusters in enumerate(num_of_clusters):
                    print(f'Running epoch {i}, {j}, {k}, {l}')
                    time_taken, average_guesses, win_rate, guesses = rl_cluster_2(learning_rate=alpha, exploration_rate=epsilon, shrinkage_factor=gamma, number_of_cluster=num_clusters, num_simulations=100)
                    df_3.loc[len(df_3)] = [alpha, epsilon, gamma, num_clusters, time_taken, average_guesses, win_rate]

    df_3.to_csv('grid_search_results/results_cluster_2.csv', index=False)
    df_3 = df_3.sort_values(by=['win_rate', 'average_guesses', 'time_taken'], ascending=[False, True, True])
    print(df_3.iloc[0])

# Running the grid-search functions in cpu parallel
def run_cpu_tasks_in_parallel(tasks):
    running_tasks = [Process(target=task) for task in tasks]
    for running_task in running_tasks:
        running_task.start()
    for running_task in running_tasks:
        running_task.join()

In [ ]:
if run_grid_search == True:
    run_cpu_tasks_in_parallel([func_1, func_2, func_3])

df_1 = pd.read_csv('grid_search_results/results_base.csv')
df_2 = pd.read_csv('grid_search_results/results_cluster.csv')
df_3 = pd.read_csv('grid_search_results/results_cluster_2.csv')

df_1 = df_1.sort_values(by=['win_rate', 'average_guesses', 'time_taken'], ascending=[False, True, True])
df_2 = df_2.sort_values(by=['win_rate', 'average_guesses', 'time_taken'], ascending=[False, True, True])
df_3 = df_3.sort_values(by=['win_rate', 'average_guesses', 'time_taken'], ascending=[False, True, True])

print(df_1.iloc[0])
print(df_2.iloc[0])
print(df_3.iloc[0])

# Evaluation 

Run each of the 5 models, using the default hyper-parameter values based off the grid-search, over 10000 simulations.

In [ ]:
import random
import numpy as np 
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from math import log10, floor
from scipy.stats import t
import matplotlib.pyplot as plt
from models.wordle_base_15k import run_simulations as rl_base
from models.wordle_cluster_15k import run_simulations as rl_cluster
from models.wordle_cluster_2k import run_simulations as rl_cluster_2
from models.wordle_greedy_search_15k import run_simulations as greed_search
from models.wordle_greedy_search_2k import run_simulations as greed_search_2

Running the 10000 simulations per model.

In [ ]:
# Running 10000 simulations for each of the 5 models and saving the results
run = False
if run:
    results = pd.DataFrame(columns=['time_taken', 'average_guesses', 'win_rate'])

    for i in tqdm(range(5)):
        if i == 0:
            time_taken, average_guesses, win_rate, guesses = rl_base(learning_rate=0.1, exploration_rate=0.8, shrinkage_factor=0.8, num_simulations=10000)
            base_guesses = guesses
            results.loc[0] = [time_taken, average_guesses, win_rate]
        elif i == 1:
            time_taken, average_guesses, win_rate, guesses = rl_cluster(learning_rate=0.1, exploration_rate=0.5, number_of_cluster=6 ,shrinkage_factor=0.9, num_simulations=10000)
            cluster_guesses = guesses
            results.loc[1] = [time_taken, average_guesses, win_rate]
        elif i == 2:
            time_taken, average_guesses, win_rate, guesses = rl_cluster_2(learning_rate=0.001, exploration_rate=0.9, number_of_cluster=9, shrinkage_factor=0.9, num_simulations=10000)
            cluster_2_guesses = guesses
            results.loc[2] = [time_taken, average_guesses, win_rate]
        elif i == 3:
            time_taken, average_guesses, win_rate, guesses = greed_search(num_simulations=10000)
            greedy_search_guesses = guesses
            results.loc[3] = [time_taken, average_guesses, win_rate]
        elif i == 4:
            time_taken, average_guesses, win_rate, guesses = greed_search_2(num_simulations=10000)
            greedy_search_2_guesses = guesses
            results.loc[4] = [time_taken, average_guesses, win_rate]

    results.to_csv('evaluation_results/results.csv')
    np.save('evaluation_results/base_guesses.npy', base_guesses)
    np.save('evaluation_results/cluster_guesses.npy', cluster_guesses)
    np.save('evaluation_results/cluster_2_guesses.npy', cluster_2_guesses)
    np.save('evaluation_results/greedy_search_guesses.npy', greedy_search_guesses)
    np.save('evaluation_results/greedy_search_2_guesses.npy', greedy_search_2_guesses)

Loading the saved results.

In [ ]:
# Load the results
results = pd.read_csv('evaluation_results/results.csv')
base_guesses = np.load('evaluation_results/base_guesses.npy')
cluster_guesses = np.load('evaluation_results/cluster_guesses.npy')
cluster_2_guesses = np.load('evaluation_results/cluster_2_guesses.npy')
greedy_search_guesses = np.load('evaluation_results/greedy_search_guesses.npy')
greedy_search_2_guesses = np.load('evaluation_results/greedy_search_2_guesses.npy')

Checking for burn-in period and if the Q-learning reduces the guesses.

In [ ]:
# Plotting the guesses over time to check for burn-in period and to check if the Q-learning reduces the guesses. 
epochs = np.arange(10000)
fig, ax = plt.subplots(3, 2, figsize=(10, 10))
ax[0, 0].bar(epochs, base_guesses, color='blue')
ax[0, 0].set_title('RL Base 15k Words')
ax[1, 0].bar(epochs, cluster_guesses, color='blue')
ax[1, 0].set_title('RL with Cluster 15k Words')
ax[1, 1].bar(epochs, cluster_2_guesses, color='blue')
ax[1, 1].set_title('RL with Cluster 2k Words')
ax[2, 0].bar(epochs, greedy_search_guesses, color='blue')
ax[2, 0].set_title('Greedy Search 15k Words')
ax[2, 1].bar(epochs, greedy_search_2_guesses, color='blue')
ax[2, 1].set_title('Greedy Search 2k Words')
ax[0, 1].set_axis_off()
fig.supxlabel('Run Index')
fig.supylabel('Number of Guesses required to get goal word')

Do the batch-avg calculations with burn-in period of 1000 aka remove first batch for average guesses.

In [ ]:
# Batch-averaging calculations with burn-in period of 1000
def batch_calc(guesses:np.ndarray): 
    guesses_mean = []
    for i in range(0, len(guesses), 1000):
        guesses_mean.append(np.mean(guesses[i:i+1000]))
    guesses_mean = np.array(guesses_mean)
    guesses_mean = guesses_mean[1:]
    return guesses_mean

# Round to nearest 3sf
def round_sig(x, sig=3):
    return round(x, sig-int(floor(log10(abs(x))))-1)

# Find the confidence interval for the batch-averaged guesses
def calculate_t_test_CI(values, confidence:int):
    dof = len(values) - 1
    conf = confidence
    t_crit = np.abs(t.ppf((1-confidence)/2,dof))
    mean = np.mean(values)
    variance = np.var(values)
    return (round((mean - t_crit * (variance / np.sqrt(dof))),3), round((mean + t_crit * (variance / np.sqrt(dof))),3))

# Do the batch-averaging calculations
base_avg_guesses = batch_calc(base_guesses)
cluster_avg_guesses = batch_calc(cluster_guesses)
cluster_2_avg_guesses = batch_calc(cluster_2_guesses)
greedy_search_avg_guesses = batch_calc(greedy_search_guesses)
greedy_search_2_avg_guesses = batch_calc(greedy_search_2_guesses)

# Plot the batch-averaged guesses
sns.lineplot(x=range(len(base_avg_guesses)), y=base_avg_guesses, marker='o', markersize=5, linewidth=1, label='RL base with 15k words')
sns.lineplot(x=range(len(cluster_avg_guesses)), y=cluster_avg_guesses, marker='o', markersize=5, linewidth=1, label='RL with cluster 15k words')
sns.lineplot(x=range(len(cluster_2_avg_guesses)), y=cluster_2_avg_guesses, marker='o', markersize=5, linewidth=1, label='RL with cluster 2k words')
sns.lineplot(x=range(len(greedy_search_avg_guesses)), y=greedy_search_avg_guesses, marker='o', markersize=5, linewidth=1, label='Greedy Search 15k words')
sns.lineplot(x=range(len(greedy_search_2_avg_guesses)), y=greedy_search_2_avg_guesses, marker='o', markersize=5, linewidth=1, label='Greedy Search 2k words')

for i in range(len(base_avg_guesses)):
    plt.text(i, base_avg_guesses[i], str(base_avg_guesses[i]), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, cluster_avg_guesses[i], str(cluster_avg_guesses[i]), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, cluster_2_avg_guesses[i], str(cluster_2_avg_guesses[i]), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, greedy_search_avg_guesses[i], str(greedy_search_avg_guesses[i]), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, greedy_search_2_avg_guesses[i], str(greedy_search_2_avg_guesses[i]), horizontalalignment='center', verticalalignment='bottom')
    
plt.title("Average guesses per 1000 simulations")
plt.xlabel("Batch number (1000 simulations per batch)")
plt.ylabel("Average guesses")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Command-line printing of the confidence interval for the batch-averaged guesses, and the average guesses mean and variance
base_avg_guesses_mean = round_sig(np.mean(base_avg_guesses),3)
cluster_avg_guesses_mean = round_sig(np.mean(cluster_avg_guesses),3)
cluster_2_avg_guesses_mean = round_sig(np.mean(cluster_2_avg_guesses),3)
greedy_search_avg_guesses_mean = round_sig(np.mean(greedy_search_avg_guesses),3)
greedy_search_2_avg_guesses_mean = round_sig(np.mean(greedy_search_2_avg_guesses),3)

base_avg_guesses_variance = round_sig(np.var(base_avg_guesses),3)
cluster_avg_guesses_variance = round_sig(np.var(cluster_avg_guesses),3)
cluster_2_avg_guesses_variance = round_sig(np.var(cluster_2_avg_guesses),3)
greedy_search_avg_guesses_variance = round_sig(np.var(greedy_search_avg_guesses),3)
greedy_search_2_avg_guesses_variance = round_sig(np.var(greedy_search_2_avg_guesses),3)

base_avg_guesses_CI = calculate_t_test_CI(base_avg_guesses, 0.95)
cluster_avg_guesses_CI = calculate_t_test_CI(cluster_avg_guesses, 0.95)
cluster_2_avg_guesses_CI = calculate_t_test_CI(cluster_2_avg_guesses, 0.95)
greedy_search_avg_guesses_CI = calculate_t_test_CI(greedy_search_avg_guesses, 0.95)
greedy_search_2_avg_guesses_CI = calculate_t_test_CI(greedy_search_2_avg_guesses, 0.95)

print(f'Steady state average guesses for RL base with 15k words is {base_avg_guesses_mean}, its variance is {base_avg_guesses_variance} and 95% confidence interval of the average guess is {base_avg_guesses_CI}.')
print(f'Steady state average guesses for RL with clustering & 15k words is {cluster_avg_guesses_mean}, its variance is {cluster_avg_guesses_variance} and 95% confidence interval of the average guess is {cluster_avg_guesses_CI}.')
print(f'Steady state average guesses for RL with clustering & 2k words is {cluster_2_avg_guesses_mean}, its variance is {cluster_2_avg_guesses_variance} and 95% confidence interval of the average guess is {cluster_2_avg_guesses_CI}.')
print(f'Steady state average guesses for Greedy search with 15k words is {greedy_search_avg_guesses_mean}, its variance is {greedy_search_avg_guesses_variance} and 95% confidence interval of the average guess is {greedy_search_avg_guesses_CI}.')
print(f'Steady state average guesses for Greedy search with 2k words is {greedy_search_2_avg_guesses_mean}, its variance is {greedy_search_2_avg_guesses_variance} and 95% confidence interval of the average guess is {greedy_search_2_avg_guesses_CI}.')

Do the batch-avg calculations with burn-in period of 1000 aka remove first batch for average win rate.

In [ ]:
# Get the winrate for each run
def calculate_win_rate(batch_len:int, guesses):
    win_rate = (batch_len-np.sum(guesses>6))/batch_len*100
    return win_rate

# Batch-averaging calculations with burn-in period of 1000
def batch_calc(guesses:np.ndarray): 
    win_rate = []
    for i in range(0, len(guesses), 1000):
        win_rate.append(calculate_win_rate(1000,guesses[i:i+1000]))
    win_rate = np.array(win_rate)
    win_rate = win_rate[1:]
    return win_rate

# Round to nearest 3sf
def round_sig(x, sig=3):
    return round(x, sig-int(floor(log10(abs(x))))-1)

# Find the confidence interval for the batch-averaged guesses
def calculate_t_test_CI(values, confidence:int):
    dof = len(values) - 1
    conf = confidence
    t_crit = np.abs(t.ppf((1-confidence)/2,dof))
    mean = np.mean(values)
    variance = np.var(values)
    return (round((mean - t_crit * (variance / np.sqrt(dof))),1), round((mean + t_crit * (variance / np.sqrt(dof))),1))

# Do the batch-averaging calculations
base_avg_win_rates = batch_calc(base_guesses)
cluster_avg_win_rates = batch_calc(cluster_guesses)
cluster_2_avg_win_rates = batch_calc(cluster_2_guesses)
greedy_search_avg_win_rates = batch_calc(greedy_search_guesses)
greedy_search_2_avg_win_rates = batch_calc(greedy_search_2_guesses)

# Plot the winrate for each batch
sns.lineplot(x=range(len(base_avg_win_rates)), y=base_avg_win_rates, marker='o', markersize=5, linewidth=1, label='RL base with 15k words')
sns.lineplot(x=range(len(cluster_avg_win_rates)), y=cluster_avg_win_rates, marker='o', markersize=5, linewidth=1, label='RL with clustering & 15k words')
sns.lineplot(x=range(len(cluster_2_avg_win_rates)), y=cluster_2_avg_win_rates, marker='o', markersize=5, linewidth=1, label='RL with clustering & 2k words')
sns.lineplot(x=range(len(greedy_search_avg_win_rates)), y=greedy_search_avg_win_rates, marker='o', markersize=5, linewidth=1, label='Greedy search with 15k words')
sns.lineplot(x=range(len(greedy_search_2_avg_win_rates)), y=greedy_search_2_avg_win_rates, marker='o', markersize=5, linewidth=1, label='Greedy search with 2k words')

for i in range(len(base_avg_win_rates)):
    plt.text(i, base_avg_win_rates[i], str(round(base_avg_win_rates[i],3)), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, cluster_avg_win_rates[i], str(round(cluster_avg_win_rates[i],3)), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, cluster_2_avg_win_rates[i], str(round(cluster_2_avg_win_rates[i],3)), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, greedy_search_avg_win_rates[i], str(round(greedy_search_avg_win_rates[i],3)), horizontalalignment='center', verticalalignment='bottom')
    plt.text(i, greedy_search_2_avg_win_rates[i], str(round(greedy_search_2_avg_win_rates[i],3)), horizontalalignment='center', verticalalignment='bottom')
    
plt.title("Average win-rate per 1000 simulations")
plt.xlabel("Batch number (1000 simulations per batch)")
plt.ylabel("Average win-rate")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# Command-line printing of the confidence interval for the batch-averaged win-rate, and the average win-rate mean and variance
base_avg_win_rates_mean = round_sig(np.mean(base_avg_win_rates),3)
cluster_avg_win_rates_mean = round_sig(np.mean(cluster_avg_win_rates),3)
cluster_2_avg_win_rates_mean = round_sig(np.mean(cluster_2_avg_win_rates),3)
greedy_search_avg_win_rates_mean = round_sig(np.mean(greedy_search_avg_win_rates),3)
greedy_search_2_avg_win_rates_mean = round_sig(np.mean(greedy_search_2_avg_win_rates),3)

base_avg_win_rates_variance = round_sig(np.var(base_avg_win_rates),3)
cluster_avg_win_rates_variance = round_sig(np.var(cluster_avg_win_rates),3)
cluster_2_avg_win_rates_variance = round_sig(np.var(cluster_2_avg_win_rates),3)
greedy_search_avg_win_rates_variance = round_sig(np.var(greedy_search_avg_win_rates),3)
greedy_search_2_avg_win_rates_variance = round_sig(np.var(greedy_search_2_avg_win_rates),3)

base_avg_win_rates_CI = calculate_t_test_CI(base_avg_win_rates, 0.95)
cluster_avg_win_rates_CI = calculate_t_test_CI(cluster_avg_win_rates, 0.95)
cluster_2_avg_win_rates_CI = calculate_t_test_CI(cluster_2_avg_win_rates, 0.95)
greedy_search_avg_win_rates_CI = calculate_t_test_CI(greedy_search_avg_win_rates, 0.95)
greedy_search_2_avg_win_rates_CI = calculate_t_test_CI(greedy_search_2_avg_win_rates, 0.95)

print(f'Steady state average guesses for RL base with 15k words is {base_avg_win_rates_mean}, its variance is {base_avg_win_rates_variance} and 95% confidence interval of the average guess is {base_avg_win_rates_CI}.')
print(f'Steady state average guesses for RL with clustering & 15k words is {cluster_avg_win_rates_mean}, its variance is {cluster_avg_win_rates_variance} and 95% confidence interval of the average guess is {cluster_avg_win_rates_CI}.')
print(f'Steady state average guesses for RL with clustering & 2k words is {cluster_2_avg_win_rates_mean}, its variance is {cluster_2_avg_win_rates_variance} and 95% confidence interval of the average guess is {cluster_2_avg_win_rates_CI}.')
print(f'Steady state average guesses for Greedy search with 15k words is {greedy_search_avg_win_rates_mean}, its variance is {greedy_search_avg_win_rates_variance} and 95% confidence interval of the average guess is {greedy_search_avg_win_rates_CI}.')
print(f'Steady state average guesses for Greedy search with 2k words is {greedy_search_2_avg_win_rates_mean}, its variance is {greedy_search_2_avg_win_rates_variance} and 95% confidence interval of the average guess is {greedy_search_2_avg_win_rates_CI}.')

In [ ]:
results['time_taken']

Imbalanced training.

In [ ]:
goal_words = []
with open('models/goal_words.txt', 'r') as file:
    for word in file:
        goal_words.append(word.strip('\n').upper())

# Simulate our random selection of goal word for the 10000 runs
words = {}
for i in range(10000):
    word = goal_words[int(np.random.uniform(0,2308))]
    if word in words:
        words[word] += 1
    else:
        words[word] = 1

# Do a count of the number of times each word was selected and plot
df = pd.DataFrame(words.items(), columns=['word', 'count'])
df.sort_values(by='count', ascending=False, inplace=True)
df = df['count'].value_counts()

df.plot(kind='bar')
plt.title("Distribution of goal words used across 10000 runs")
plt.xlabel("Number of times used as goal words")
plt.ylabel("Count of goal words")